## VSS based SSS

In [118]:
"""
The following Python implementation of Shamir's Secret Sharing is
released into the Public Domain under the terms of CC0 and OWFa:
https://creativecommons.org/publicdomain/zero/1.0/
http://www.openwebfoundation.org/legal/the-owf-1-0-agreements/owfa-1-0

See the bottom few lines for usage. Tested on Python 2 and 3.
"""

from __future__ import division
from __future__ import print_function

import random
import functools

# 12th Mersenne Prime
# (for this application we want a known prime number as close as
# possible to our security level; e.g.  desired security level of 128
# bits -- too large and all the ciphertext is large; too small and
# security is compromised)
_PRIME = 100003 #2 ** 127 - 1
# 13th Mersenne Prime is 2**521 - 1

_RINT = functools.partial(random.SystemRandom().randint, 0)

def _eval_at(poly, x, prime):
    """Evaluates polynomial (coefficient tuple) at x, used to generate a
    shamir pool in make_random_shares below.
    """
    accum = 0
    for coeff in reversed(poly):
        accum *= x
        accum += coeff
        accum %= prime
    return accum

def make_random_shares(secret, minimum, shares, prime=_PRIME):
    """
    Generates a random shamir pool for a given secret, returns share points.
    """
    if minimum > shares:
        raise ValueError("Pool secret would be irrecoverable.")
    poly = [secret] + [_RINT(prime - 1) for i in range(minimum - 1)]
    points = [(i, _eval_at(poly, i, prime))
              for i in range(1, shares + 1)]
    return points

def _extended_gcd(a, b):
    """
    Division in integers modulus p means finding the inverse of the
    denominator modulo p and then multiplying the numerator by this
    inverse (Note: inverse of A is B such that A*B % p == 1) this can
    be computed via extended Euclidean algorithm
    http://en.wikipedia.org/wiki/Modular_multiplicative_inverse#Computation
    """
    x = 0
    last_x = 1
    y = 1
    last_y = 0
    while b != 0:
        quot = a // b
        a, b = b, a % b
        x, last_x = last_x - quot * x, x
        y, last_y = last_y - quot * y, y
    return last_x, last_y

def _divmod(num, den, p):
    """Compute num / den modulo prime p

    To explain what this means, the return value will be such that
    the following is true: den * _divmod(num, den, p) % p == num
    """
    inv, _ = _extended_gcd(den, p)
    return num * inv

def _lagrange_interpolate(x, x_s, y_s, p):
    """
    Find the y-value for the given x, given n (x, y) points;
    k points will define a polynomial of up to kth order.
    """
    k = len(x_s)
    assert k == len(set(x_s)), "points must be distinct"
    def PI(vals):  # upper-case PI -- product of inputs
        accum = 1
        for v in vals:
            accum *= v
        return accum
    nums = []  # avoid inexact division
    dens = []
    for i in range(k):
        others = list(x_s)
        cur = others.pop(i)
        nums.append(PI(x - o for o in others))
        dens.append(PI(cur - o for o in others))
    den = PI(dens)
    num = sum([_divmod(nums[i] * den * y_s[i] % p, dens[i], p)
               for i in range(k)])
    return (_divmod(num, den, p) + p) % p

def recover_secret(shares, prime=_PRIME):
    """
    Recover the secret from share points
    (x, y points on the polynomial).
    """
    if len(shares) < 3:
        raise ValueError("need at least three shares")
    x_s, y_s = zip(*shares)
    return _lagrange_interpolate(0, x_s, y_s, prime)

def main():
    """Main function"""
    secret = 1234
    shares = make_random_shares(secret, minimum=3, shares=6)

    print('Secret:',secret)
    print('Shares:')
    if shares:
        for share in shares:
            print('  ', share)

    print('Secret recovered from minimum subset of shares:',
          recover_secret(shares[:3]))
    print('Secret recovered from a different minimum subset of shares: ',
          recover_secret(shares[-3:]))

if __name__ == '__main__':
    main()

Secret: 1234
Shares:
   (1, 95469)
   (2, 96555)
   (3, 4492)
   (4, 19286)
   (5, 40934)
   (6, 69436)
Secret recovered from minimum subset of shares: 1234
Secret recovered from a different minimum subset of shares:  1234


In [119]:
import random
import math
from decimal import Decimal
import time

def isprime(n):
    if n == 2:
        return True
    if n == 1 or n % 2 == 0:
        return False
    i = 3
    while i <= math.sqrt(n):
        if n % i == 0:
            return False
        i = i + 2
    return True


def initial(Z_lower=100):
    # generate q bigger than z_lower
    q = Z_lower
    while True:
        if isprime(q):
            break
        else:
            q = q + 1
    print("q = " + str(q))
    print("\nq is prime\n")

    # Find p and r
    r = 1
    while True:
        p = r * q + 1
        if isprime(p):
            print("r = " + str(r))
            print("p = " + str(p))
            print("\np is prime\n")
            break
        r = r + 1

    # Compute elements of Z_p*
    Z_p_star = []
    for i in range(0, p):
        if (math.gcd(i, p) == 1):
            Z_p_star.append(i)

    # print("Z_p* = ")
    # print(Z_p_star) # , len(Z_p_star) same length, i.e. range(p)

    # Compute elements of G = {h^r mod p | h in Z_p*}
    G = []
    for i in Z_p_star:
        G.append(i ** r % p)

    G = list(set(G))
    G.sort()
    # print("\nG = ")
    # print(G)
    # print("Order of G is " + str(len(G)) + ". This must be equal to q.")

    # Since the order of G is prime, any element of G except 1 is a generator
    g = random.choice(list(filter(lambda g: g != 1, G)))
    print("\ng = " + str(g) + "\n")

    return p, q, r, g

def generate_shares(n, t, secret, p, q, r, g, idxs):
    if secret==0: secret=1 # one exception after quantization
    assert secret >= 1 and secret <= q, "secret not in range"

    FIELD_SIZE = q
    coefficients = coeff(t, secret, FIELD_SIZE)
    users = list(idxs) # users are to recieve the shares
    assert n==len(users), "these two number should be identical"

    shares = []
    for i in users:
        f_i = f(i, coefficients, q)
        shares.append((i, f_i))

    commitments = commitment(coefficients, g, p)
    verifications = []
    for i in users:
        # check1 = g ** shares[i-1][1] % p
        # check1 = g ** share_ith(shares, i) % p
        check1 = quick_pow(g, share_ith(shares, i), p)
        check2 = verification(g, commitments, i, p)
        verifications.append(check2)
        if check1 == check2:
            pass
        else:
            print("checking fails with:", check1, check2)

    return shares, commitments, verifications

def share_ith(shares, i):
    for share in shares:
        if share[0] == i:
            return share[1]
    return None

def coeff(t, secret, FIELD_SIZE):
    coeff = [random.randrange(0, FIELD_SIZE) for _ in range(t - 1)]
    coeff.append(secret)  # a0 is secret
    return coeff

def f(x, coefficients, q):
    y = Decimal('0')
    for coefficient_index, coefficient_value in enumerate(coefficients[::-1]):
        y += (Decimal(str(x)) ** Decimal(str(coefficient_index)) * Decimal(str(coefficient_value)))
        y = Decimal(int(y)%q)
    return int(y)


def commitment(coefficients, g, p):
    commitments = []
    for coefficient_index, coefficient_value in enumerate(coefficients[::-1]):
        c = g ** coefficient_value % p
        # c = quick_pow(g,coefficient_value,p)
        commitments.append(c)
    return commitments


def verification(g, commitments, i, p):
    v = 1
    for k, c in enumerate(commitments):
        v = v * (c) ** (i ** k) % p
        # v = v * quick_pow(c,i ** k,p) % p
    return v


def quick_pow(a, b, q):  # compute a^b mod q, in a faster way
    temp = 1
    for i in range(1, b + 1):
        temp = temp * a % q
    return temp % q


def reconstruct_secret(pool, q):
    x_s,y_s = [],[]
    for share in pool:
        x_s.append(share[0])
        y_s.append(share[1])
    out = _lagrange_interpolate(0, x_s, y_s, q)
    return out

def _lagrange_interpolate(x, x_s, y_s, p):
    """
    Find the y-value for the given x, given n (x, y) points;
    k points will define a polynomial of up to kth order.
    """
    k = len(x_s)
    assert k == len(set(x_s)), "points must be distinct"
    def PI(vals):  # upper-case PI -- product of inputs
        accum = 1
        for v in vals:
            accum *= v
        return accum
    nums = []  # avoid inexact division
    dens = []
    for i in range(k):
        others = list(x_s)
        cur = others.pop(i)
        nums.append(PI(x - o for o in others))
        dens.append(PI(cur - o for o in others))
    den = PI(dens)
    num = sum([_divmod(nums[i] * den * y_s[i] % p, dens[i], p)
               for i in range(k)])
    return (_divmod(num, den, p) + p) % p

def _extended_gcd(a, b):
    """
    Division in integers modulus p means finding the inverse of the
    denominator modulo p and then multiplying the numerator by this
    inverse (Note: inverse of A is B such that A*B % p == 1) this can
    be computed via extended Euclidean algorithm
    http://en.wikipedia.org/wiki/Modular_multiplicative_inverse#Computation
    """
    x = 0
    last_x = 1
    y = 1
    last_y = 0
    while b != 0:
        quot = a // b
        a, b = b, a % b
        x, last_x = last_x - quot * x, x
        y, last_y = last_y - quot * y, y
    return last_x, last_y

def _divmod(num, den, p):
    """Compute num / den modulo prime p

    To explain what this means, the return value will be such that
    the following is true: den * _divmod(num, den, p) % p == num
    """
    inv, _ = _extended_gcd(den, p)
    return num * inv


In [120]:
# Driver code
if __name__ == '__main__':
    # initialization
    time_start = time.time()
    print("========Main VSS Starts==========")
    p,q,r,g = initial(10**5)

    # Secret taken from the group Z_q* 
    t, n = 5, 10
    secret = 1243
    print(f'Original Secret: {secret}')

    # Phase I: Generation of shares
    users = [9, 3, 7, 8, 4, 6, 5, 2, 1, 10]
    print(n, t, secret, p, q, r, g, users)
    shares, commitments, verifications= generate_shares(n, t, secret, p, q, r, g, users)
    print(f'Shares: {", ".join(str(share) for share in shares)}')
    print(f'Commitments: {", ".join(str(commitment) for commitment in commitments)}')
    print(f'verifications: {", ".join(str(verification) for verification in verifications)}')

    # Phase II: Secret Reconstruction
    # Picking t shares randomly for reconstruction
    pool = random.sample(shares, t)
    print(f'Combining shares: {", ".join(str(share) for share in pool)}')
    secret_reconstructed = reconstruct_secret(pool, q)
    print("reconstruct_secret:",secret_reconstructed)

    time_end = time.time()
    print('time cost in second:', time_end-time_start)



========Main VSS Starts==========
q = 100003

q is prime

r = 42
p = 4200127

p is prime


g = 2147431

Original Secret: 1243
10 5 1243 4200127 100003 42 2147431 [9, 3, 7, 8, 4, 6, 5, 2, 1, 10]
Shares: (9, 18120), (3, 28819), (7, 93979), (8, 90164), (4, 54005), (6, 26763), (5, 62912), (2, 93326), (1, 30681), (10, 57847)
Commitments: 1761168, 2124748, 1050745, 1380806, 832437
verifications: 1190454, 4151512, 2634972, 1383553, 4013481, 3655039, 1539058, 521965, 3284121, 3424064
Combining shares: (7, 93979), (5, 62912), (10, 57847), (9, 18120), (1, 30681)
reconstruct_secret: 1243
time cost in second: 5.674405813217163


## LCC

In [121]:
def isprime(n):
    if n == 2:
        return True
    if n == 1 or n % 2 == 0:
        return False
    i = 3
    while i <= math.sqrt(n):
        if n % i == 0:
            return False
        i = i + 2
    return True


def initial(Z_lower=100):
    # generate q bigger than z_lower
    q = Z_lower
    while True:
        if isprime(q):
            break
        else:
            q = q + 1
    print("q = " + str(q))
    print("\nq is prime\n")

    # Find p and r
    r = 1
    while True:
        p = r * q + 1
        if isprime(p):
            print("r = " + str(r))
            print("p = " + str(p))
            print("\np is prime\n")
            break
        r = r + 1

    # Compute elements of Z_p*
    Z_p_star = []
    for i in range(0, p):
        if (math.gcd(i, p) == 1):
            Z_p_star.append(i)

    # print("Z_p* = ")
    # print(Z_p_star) # , len(Z_p_star) same length, i.e. range(p)

    # Compute elements of G = {h^r mod p | h in Z_p*}
    G = []
    for i in Z_p_star:
        G.append(i ** r % p)

    G = list(set(G))
    G.sort()
    # print("\nG = ")
    # print(G)
    # print("Order of G is " + str(len(G)) + ". This must be equal to q.")

    # Since the order of G is prime, any element of G except 1 is a generator
    g = random.choice(list(filter(lambda g: g != 1, G)))
    print("\ng = " + str(g) + "\n")

    return p, q, r, g


def generate_shares(N, T, K, secrets, p, q, r, g, alphas, betas):
    for secret in secrets:
        assert secret >= 1 and secret <= q, "secret not in range"

    FIELD_SIZE = q
    noises = [random.randrange(0, FIELD_SIZE) for _ in range(T)]
    
    shares = []
    for alpha in alphas:
        # y = f(alpha, secrets, noises, betas, q)
        y = _lagrange_interpolate(alpha, betas, secrets + noises, q)
        shares.append((alpha, y))

#     commitments = commitment(coefficients, g, p)
#     verifications = []
#     for i in users:
#         # check1 = g ** shares[i-1][1] % p
#         # check1 = g ** share_ith(shares, i) % p
#         check1 = quick_pow(g, share_ith(shares, i), p)
#         check2 = verification(g, commitments, i, p)
#         verifications.append(check2)
#         if check1 == check2:
#             pass
#         else:
#             print("checking fails with:", check1, check2)

#     return shares, commitments, verifications
    return shares,None,None


def f(x, secrets, zs, betas, q):
    y = Decimal('0')
    for i, s in enumerate(secrets):
        beta_i = Decimal(str(betas[i]))
        prod = Decimal('1')
        for beta in betas:
            if beta != int(beta_i):
                prod *= (Decimal(str(x))-Decimal(str(beta)))/(beta_i-Decimal(str(beta)))
            else:pass
        y += Decimal(str(s)) * prod
        y = Decimal(round(y)%q)
    k = len(secrets)
    for j, z in enumerate(zs):
        beta_j = Decimal(str(betas[k + j]))
        prod = Decimal('1')
        for beta in betas:
            if beta != int(beta_j):
                prod *= (Decimal(str(x)) - Decimal(str(beta))) / (beta_j - Decimal(str(beta)))
            else:pass
        y += Decimal(str(z)) * prod 
        y = Decimal(round(y)%q)
    return int(y)

def reconstruct_secret(pool, q, betas, K):
    out = []
    x_s,y_s = [],[]
    for share in pool:
        x_s.append(share[0])
        y_s.append(share[1])
    for k in range(K):
        beta = betas[k]
        # out.append(f_rec(beta,pool,q))
        out.append(_lagrange_interpolate(beta, x_s, y_s, q))
    return out

def f_rec(x, pool, q):
    y = Decimal('0')
    for i, s in enumerate(pool):
        alpha_i, share_i = pool[i]
        prod = Decimal('1')
        for share in pool:
            if share_i != share[1]:
                prod *= (Decimal(str(x))-Decimal(str(share[0]))) / (Decimal(str(alpha_i))-Decimal(str(share[0])))
            else:pass
        y += Decimal(str(share_i)) * prod
        y = Decimal(int(y)%q)
    return int(y)

def _lagrange_interpolate(x, x_s, y_s, p):
    """
    Find the y-value for the given x, given n (x, y) points;
    k points will define a polynomial of up to kth order.
    """
    k = len(x_s)
    assert k == len(set(x_s)), "points must be distinct"
    def PI(vals):  # upper-case PI -- product of inputs
        accum = 1
        for v in vals:
            accum *= v
        return accum
    nums = []  # avoid inexact division
    dens = []
    for i in range(k):
        others = list(x_s)
        cur = others.pop(i)
        nums.append(PI(x - o for o in others))
        dens.append(PI(cur - o for o in others))
    den = PI(dens)
    num = sum([_divmod(nums[i] * den * y_s[i] % p, dens[i], p)
               for i in range(k)])
    return (_divmod(num, den, p) + p) % p

def _extended_gcd(a, b):
    """
    Division in integers modulus p means finding the inverse of the
    denominator modulo p and then multiplying the numerator by this
    inverse (Note: inverse of A is B such that A*B % p == 1) this can
    be computed via extended Euclidean algorithm
    http://en.wikipedia.org/wiki/Modular_multiplicative_inverse#Computation
    """
    x = 0
    last_x = 1
    y = 1
    last_y = 0
    while b != 0:
        quot = a // b
        a, b = b, a % b
        x, last_x = last_x - quot * x, x
        y, last_y = last_y - quot * y, y
    return last_x, last_y

def _divmod(num, den, p):
    """Compute num / den modulo prime p

    To explain what this means, the return value will be such that
    the following is true: den * _divmod(num, den, p) % p == num
    """
    inv, _ = _extended_gcd(den, p)
    return num * inv

In [122]:

# initialization
time_start = time.time()
print("========Main LCC Starts==========")
p,q,r,g = initial(10**5)

# Secret taken from the group Z_q* 
T, N, K = 3, 15, 3
secrets = [422,378,1101]
print(f'Original Secret: {secrets}')

# Phase I: Generation of shares
alphas = list(range(1, 1+N))
betas = list(range(1+N, N+K+T+1))
shares, commitments, verifications= generate_shares(N, T, K, secrets, p, q, r, g, alphas, betas)
print(f'Shares: {", ".join(str(share) for share in shares)}')
# print(f'Commitments: {", ".join(str(commitment) for commitment in commitments)}')
# print(f'verifications: {", ".join(str(verification) for verification in verifications)}')

# Phase II: Secret Reconstruction
# Picking t shares randomly for reconstruction
pool = random.sample(shares, T+K)
print(f'Combining shares: {", ".join(str(share) for share in pool)}')
secret_reconstructed = reconstruct_secret(pool, q, betas, K)
print("reconstruct_secret:",secret_reconstructed)

time_end = time.time()
print('time cost in second:', time_end-time_start)

========Main LCC Starts==========
q = 100003

q is prime

r = 42
p = 4200127

p is prime


g = 2450215

Original Secret: [422, 378, 1101]
Shares: (1, 96927), (2, 39211), (3, 81959), (4, 93396), (5, 89295), (6, 16014), (7, 33548), (8, 98560), (9, 47427), (10, 79298), (11, 39116), (12, 679), (13, 49677), (14, 66729), (15, 10435)
Combining shares: (12, 679), (13, 49677), (3, 81959), (8, 98560), (10, 79298), (1, 96927)
reconstruct_secret: [422, 378, 1101]
time cost in second: 5.603201150894165


In [117]:
a = 1009
a % 101, Decimal(str(a)) % 101

(100, Decimal('100'))